In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
from MCForecastTools import MCSimulation
import json

%matplotlib inline

In [2]:
?MCSimulation

Init signature:
MCSimulation(
    portfolio_data,
    weights='',
    num_simulation=1000,
    num_trading_days=252,
)
Docstring:     
A Python class for runnning Monte Carlo simulation on portfolio price data. 

...

Attributes
----------
portfolio_data : pandas.DataFrame
    portfolio dataframe
weights: list(float)
    portfolio investment breakdown
nSim: int
    number of samples in simulation
nTrading: int
    number of trading days to simulate
simulated_return : pandas.DataFrame
    Simulated data from Monte Carlo
confidence_interval : pandas.Series
    the 95% confidence intervals for simulated final cumulative returns
    
Init docstring:
Constructs all the necessary attributes for the MCSimulation object.

Parameters
----------
portfolio_data: pandas.DataFrame
    DataFrame containing stock price information from Alpaca API
weights: list(float)
    A list fractions representing percentage of total investment per stock. DEFAULT: Equal distribution
num_simulation: int
    Number 

In [3]:
load_dotenv()

alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")

In [4]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
today = pd.Timestamp("2020-06-01", tz="America/New_York").isoformat()


In [6]:
tickers = ["SPY", "AGG"]

In [7]:
timeframe = "1Day"

In [8]:
start_date = pd.Timestamp("1990-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()

In [9]:
# Get closing prices for SPY & AGG for past 31 years
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_portfolio

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,108.54,108.8400,108.43,108.82,4259986,15562,108.754152,AGG
2015-12-02 05:00:00+00:00,108.73,108.7799,108.61,108.67,2462787,11581,108.684692,AGG
2015-12-03 05:00:00+00:00,108.41,108.4100,107.81,107.89,4634020,16801,108.040315,AGG
2015-12-04 05:00:00+00:00,108.05,108.3000,108.00,108.24,2182057,9796,108.192232,AGG
2015-12-07 05:00:00+00:00,108.30,108.5800,108.23,108.40,2143773,9104,108.460067,AGG
...,...,...,...,...,...,...,...,...
2021-12-27 05:00:00+00:00,472.09,477.3100,472.01,477.26,56689618,379424,475.278749,SPY
2021-12-28 05:00:00+00:00,477.80,478.8100,476.06,476.87,47190464,371806,477.232285,SPY
2021-12-29 05:00:00+00:00,477.04,478.5600,475.92,477.48,54405126,345000,477.260209,SPY


In [10]:
# Get closing prices for SPY & AGG for past 31 years

SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol', axis=1)
AGG = df_portfolio[df_portfolio['symbol']=='AGG'].drop('symbol', axis=1)

df_portfolio_year = pd.concat([SPY, AGG],axis=1, keys=['SPY','AGG'])

# set in ascending order (past to present)
df_portfolio_year = df_portfolio_year.sort_index()

df_portfolio_year.head()

SPY                                         \
                             open      high       low   close     volume   
timestamp                                                                  
2015-12-01 05:00:00+00:00  209.42  210.8200  209.1100  210.68   97858418   
2015-12-02 05:00:00+00:00  210.60  211.0000  208.2300  208.54  108069059   
2015-12-03 05:00:00+00:00  208.90  209.1500  204.7511  205.58  166224154   
2015-12-04 05:00:00+00:00  206.10  209.9700  205.9300  209.66  192878747   
2015-12-07 05:00:00+00:00  209.20  209.7295  207.2000  208.27  102027111   

                                                      AGG                    \
                          trade_count        vwap    open      high     low   
timestamp                                                                     
2015-12-01 05:00:00+00:00      337780  209.921410  108.54  108.8400  108.43   
2015-12-02 05:00:00+00:00      367013  209.563055  108.73  108.7799  108.61   
2015-12-03 05:00:00+00:00      546768  206.878936  108.41  108.4100  107.81   
2015-12-04 05:00:00+00:00      556731  208.178631  108.05  108.3000  108.00   
2015-12-07 05:00:00+00:00      374705  208.276128  108.30  108.5800  108.23   

                                                                    
                            close   volume trade_count        vwap  
timestamp                                                           
2015-12-01 05:00:00+00:00  108.82  4259986       15562  108.754152  
2015-12-02 05:00:00+00:00  108.67  2462787       11581  108.684692  
2015-12-03 05:00:00+00:00  107.89  4634020       16801  108.040315  
2015-12-04 05:00:00+00:00  108.24  2182057        9796  108.192232  
2015-12-07 05:00:00+00:00  108.40  2143773        9104  108.460067

In [11]:
daily_returns = df_portfolio_year.pct_change().dropna()
daily_returns.head()

SPY                                          \
                               open      high       low     close    volume   
timestamp                                                                     
2015-12-02 05:00:00+00:00  0.005635  0.000854 -0.004208 -0.010158  0.104341   
2015-12-03 05:00:00+00:00 -0.008072 -0.008768 -0.016707 -0.014194  0.538129   
2015-12-04 05:00:00+00:00 -0.013404  0.003921  0.005758  0.019846  0.160353   
2015-12-07 05:00:00+00:00  0.015041 -0.001145  0.006167 -0.006630 -0.471030   
2015-12-08 05:00:00+00:00 -0.013002 -0.006868 -0.006853 -0.006146  0.013185   

                                                      AGG                      \
                          trade_count      vwap      open      high       low   
timestamp                                                                       
2015-12-02 05:00:00+00:00    0.086544 -0.001707  0.001751 -0.000552  0.001660   
2015-12-03 05:00:00+00:00    0.489778 -0.012808 -0.002943 -0.003400 -0.007366   
2015-12-04 05:00:00+00:00    0.018222  0.006282 -0.003321 -0.001015  0.001762   
2015-12-07 05:00:00+00:00   -0.326955  0.000468  0.002314  0.002585  0.002130   
2015-12-08 05:00:00+00:00    0.034899 -0.006289  0.002678  0.000184  0.001294   

                                                                     
                              close    volume trade_count      vwap  
timestamp                                                            
2015-12-02 05:00:00+00:00 -0.001378 -0.421879   -0.255815 -0.000639  
2015-12-03 05:00:00+00:00 -0.007178  0.881616    0.450738 -0.005929  
2015-12-04 05:00:00+00:00  0.003244 -0.529122   -0.416939  0.001406  
2015-12-07 05:00:00+00:00  0.001478 -0.017545   -0.070641  0.002476  
2015-12-08 05:00:00+00:00  0.000369  0.078379    0.199912 -0.000057

In [12]:
# Create a simulation object
# This portfolio will have a 80/20 split between msft and aapl set in the weight parameter
# We set the number of simulations trials to be 100
# The period over which we will simulate is the number of trading days in a year times the number of years until the child reaches college.
# for this example, the child is 8 years old (meaning 10 years until college)
higher_risk_df = MCSimulation(
    portfolio_data=df_portfolio_year,
    weights=[0.80, 0.20],
    num_simulation=100,
    num_trading_days=252*10,
)

In [13]:
higher_risk_df.portfolio_data.head()

SPY                                         \
                             open      high       low   close     volume   
timestamp                                                                  
2015-12-01 05:00:00+00:00  209.42  210.8200  209.1100  210.68   97858418   
2015-12-02 05:00:00+00:00  210.60  211.0000  208.2300  208.54  108069059   
2015-12-03 05:00:00+00:00  208.90  209.1500  204.7511  205.58  166224154   
2015-12-04 05:00:00+00:00  206.10  209.9700  205.9300  209.66  192878747   
2015-12-07 05:00:00+00:00  209.20  209.7295  207.2000  208.27  102027111   

                                                                   AGG  \
                          trade_count        vwap daily_return    open   
timestamp                                                                
2015-12-01 05:00:00+00:00      337780  209.921410          NaN  108.54   
2015-12-02 05:00:00+00:00      367013  209.563055    -0.010158  108.73   
2015-12-03 05:00:00+00:00      546768  206.878936    -0.014194  108.41   
2015-12-04 05:00:00+00:00      556731  208.178631     0.019846  108.05   
2015-12-07 05:00:00+00:00      374705  208.276128    -0.006630  108.30   

                                                                          \
                               high     low   close   volume trade_count   
timestamp                                                                  
2015-12-01 05:00:00+00:00  108.8400  108.43  108.82  4259986       15562   
2015-12-02 05:00:00+00:00  108.7799  108.61  108.67  2462787       11581   
2015-12-03 05:00:00+00:00  108.4100  107.81  107.89  4634020       16801   
2015-12-04 05:00:00+00:00  108.3000  108.00  108.24  2182057        9796   
2015-12-07 05:00:00+00:00  108.5800  108.23  108.40  2143773        9104   

                                                    
                                 vwap daily_return  
timestamp                                           
2015-12-01 05:00:00+00:00  108.754152          NaN  
2015-12-02 05:00:00+00:00  108.684692    -0.001378  
2015-12-03 05:00:00+00:00  108.040315    -0.007178  
2015-12-04 05:00:00+00:00  108.192232     0.003244  
2015-12-07 05:00:00+00:00  108.460067     0.001478

In [14]:
# Cumulative Returns of higher risk portfolio
higher_risk_df.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.994781,0.994644,1.012084,0.998377,1.008038,0.996808,1.003628,1.009939,0.995287,0.997426,...,0.993226,0.993845,0.991662,0.995497,0.992477,0.996803,0.997857,0.995225,1.004489,0.989324
2,0.979248,0.988475,1.031468,1.004181,0.995019,0.996535,1.011684,1.011945,1.004680,0.985675,...,0.985778,0.985879,0.992863,0.996234,1.003320,0.992716,1.002290,0.995172,0.997552,0.990703
3,0.982671,0.982250,1.013215,0.993394,1.008944,0.985013,0.991078,1.023937,1.004361,0.988978,...,0.983758,0.979804,1.002898,0.995903,1.013218,0.984562,1.013617,0.992671,0.994820,0.976503
4,0.983436,0.990059,1.016044,0.993317,1.009064,0.969398,0.999776,1.023772,0.990309,0.980905,...,1.001957,0.976148,0.982684,0.985280,1.005471,0.984613,1.019144,0.996729,0.994368,0.981641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,2.462966,3.622868,1.818473,5.407282,6.134069,2.697166,2.916793,3.970798,2.132839,2.963694,...,1.849156,2.561501,2.509809,5.360423,3.290510,2.399260,9.947131,3.208790,2.324260,3.159589
2517,2.456716,3.621700,1.820164,5.446421,6.187348,2.676830,2.915154,4.053378,2.119937,2.962458,...,1.845010,2.565088,2.519908,5.424316,3.282854,2.445171,9.968188,3.244639,2.344642,3.172450
2518,2.436351,3.669065,1.844161,5.452623,6.268662,2.677175,2.908272,4.103164,2.113008,3.001631,...,1.850357,2.542252,2.554192,5.420046,3.321656,2.426796,10.010525,3.292673,2.323715,3.200532
2519,2.450713,3.619910,1.837497,5.307885,6.341701,2.711834,2.933451,4.096921,2.106673,2.985098,...,1.849030,2.565155,2.571641,5.533457,3.269301,2.440181,9.979580,3.311584,2.356523,3.213416


In [15]:
# Set weights for low risk portfolio (20% SPY, 80% AGG)
low_risk_df = MCSimulation(
    portfolio_data=df_portfolio_year,
    weights=[0.20, 0.80],
    num_simulation=100,
    num_trading_days=252*10,
)

In [16]:
low_risk_df.portfolio_data.head()

SPY                                         \
                             open      high       low   close     volume   
timestamp                                                                  
2015-12-01 05:00:00+00:00  209.42  210.8200  209.1100  210.68   97858418   
2015-12-02 05:00:00+00:00  210.60  211.0000  208.2300  208.54  108069059   
2015-12-03 05:00:00+00:00  208.90  209.1500  204.7511  205.58  166224154   
2015-12-04 05:00:00+00:00  206.10  209.9700  205.9300  209.66  192878747   
2015-12-07 05:00:00+00:00  209.20  209.7295  207.2000  208.27  102027111   

                                                                   AGG  \
                          trade_count        vwap daily_return    open   
timestamp                                                                
2015-12-01 05:00:00+00:00      337780  209.921410          NaN  108.54   
2015-12-02 05:00:00+00:00      367013  209.563055    -0.010158  108.73   
2015-12-03 05:00:00+00:00      546768  206.878936    -0.014194  108.41   
2015-12-04 05:00:00+00:00      556731  208.178631     0.019846  108.05   
2015-12-07 05:00:00+00:00      374705  208.276128    -0.006630  108.30   

                                                                          \
                               high     low   close   volume trade_count   
timestamp                                                                  
2015-12-01 05:00:00+00:00  108.8400  108.43  108.82  4259986       15562   
2015-12-02 05:00:00+00:00  108.7799  108.61  108.67  2462787       11581   
2015-12-03 05:00:00+00:00  108.4100  107.81  107.89  4634020       16801   
2015-12-04 05:00:00+00:00  108.3000  108.00  108.24  2182057        9796   
2015-12-07 05:00:00+00:00  108.5800  108.23  108.40  2143773        9104   

                                                    
                                 vwap daily_return  
timestamp                                           
2015-12-01 05:00:00+00:00  108.754152          NaN  
2015-12-02 05:00:00+00:00  108.684692    -0.001378  
2015-12-03 05:00:00+00:00  108.040315    -0.007178  
2015-12-04 05:00:00+00:00  108.192232     0.003244  
2015-12-07 05:00:00+00:00  108.460067     0.001478

In [17]:
# Cumulative Returns of low risk portfolio
low_risk_df.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.006618,1.001630,0.997808,0.995127,0.998269,1.002755,1.000186,1.002152,1.004429,0.995691,...,1.001647,0.996688,1.003159,1.000313,1.006379,0.993030,0.999216,0.996190,0.994004,1.000222
2,1.003936,0.996100,0.993070,0.993864,1.000872,0.998393,1.000397,0.999586,1.007032,0.993791,...,0.996908,0.990996,1.005559,1.004131,1.011535,0.992140,0.998242,0.995715,0.991449,0.993261
3,0.999358,0.998528,0.988430,0.988172,1.000529,0.996521,1.003348,0.997668,1.011006,0.988849,...,0.996035,0.987254,1.003283,1.004551,1.010550,0.996720,1.000000,0.993738,0.989149,0.996068
4,1.001011,0.999026,0.988849,0.987268,0.995179,0.995515,1.006694,1.005112,1.008756,0.990024,...,0.995433,0.991496,1.005760,1.001560,1.017615,0.993817,0.992987,0.990042,0.989327,0.994603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,1.495717,1.074562,1.469612,1.514225,1.908006,1.359758,1.654730,1.673113,1.648639,1.459676,...,1.212711,1.556719,1.268281,1.518114,1.860381,1.753150,1.180860,1.652733,1.592275,1.717722
2517,1.494520,1.074742,1.480440,1.509375,1.910376,1.358466,1.653323,1.675113,1.644004,1.463381,...,1.213949,1.553722,1.266445,1.521490,1.846934,1.762835,1.178717,1.653078,1.596575,1.706402
2518,1.493192,1.073786,1.480870,1.511458,1.906992,1.355822,1.650481,1.669147,1.640111,1.459515,...,1.214679,1.551040,1.268900,1.521966,1.847375,1.755556,1.181556,1.652850,1.593070,1.709646
2519,1.495741,1.072156,1.477467,1.513450,1.917363,1.354525,1.654081,1.668678,1.633259,1.458783,...,1.200734,1.545405,1.275323,1.522110,1.855196,1.753412,1.179047,1.652635,1.592508,1.701707


In [18]:
agg_daily_return = AGG.pct_change()
agg_daily_return

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-02 05:00:00+00:00,0.001751,-0.000552,0.001660,-0.001378,-0.421879,-0.255815,-0.000639
2015-12-03 05:00:00+00:00,-0.002943,-0.003400,-0.007366,-0.007178,0.881616,0.450738,-0.005929
2015-12-04 05:00:00+00:00,-0.003321,-0.001015,0.001762,0.003244,-0.529122,-0.416939,0.001406
2015-12-07 05:00:00+00:00,0.002314,0.002585,0.002130,0.001478,-0.017545,-0.070641,0.002476
...,...,...,...,...,...,...,...
2021-12-27 05:00:00+00:00,-0.001225,-0.000087,0.000526,0.000525,-0.012771,-0.098702,0.000701
2021-12-28 05:00:00+00:00,0.002453,0.001312,0.000350,-0.000438,-0.033580,0.056784,0.000453
2021-12-29 05:00:00+00:00,-0.003845,-0.003495,-0.003153,-0.002977,0.128607,0.111431,-0.003318


In [19]:
agg_cumulative_return = (1 + agg_daily_return).cumprod() - 1
agg_cumulative_return

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-02 05:00:00+00:00,0.001751,-0.000552,0.001660,-0.001378,-0.421879,-0.255815,-0.000639
2015-12-03 05:00:00+00:00,-0.001198,-0.003951,-0.005718,-0.008546,0.087802,0.079617,-0.006564
2015-12-04 05:00:00+00:00,-0.004514,-0.004961,-0.003966,-0.005330,-0.487778,-0.370518,-0.005167
2015-12-07 05:00:00+00:00,-0.002211,-0.002389,-0.001845,-0.003860,-0.496765,-0.414985,-0.002704
...,...,...,...,...,...,...,...
2021-12-27 05:00:00+00:00,0.051686,0.050165,0.052568,0.049991,0.276140,-0.049415,0.050487
2021-12-28 05:00:00+00:00,0.054266,0.051544,0.052937,0.049531,0.233287,0.004562,0.050963
2021-12-29 05:00:00+00:00,0.050212,0.047868,0.049617,0.046407,0.391896,0.116502,0.047476
